In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

In [2]:
fp = '../data/full_data.csv'

In [3]:
musics = pd.read_csv(fp).iloc[:, 1:].sort_values(by = 'year')

In [4]:
musics.head()

,album,artist,id,title,year,file,mfcc_0,mfcc_1,mfcc_2,mfcc_3,...,bandwidth,centroid,rolloff,acousticness,danceability,energy,loudness,speechiness,tempo,valence
0,The Sugarhill Gang,The Sugarhill Gang,1FnxNtKli7rcB9IU4J8MEg,Rapper's Delight,1980,./data/Rapper's Delight.mp3,1.890811e-08,-3.804202e-07,-2.639254e-07,-6.431760e-08,...,2932.269422,3271.149826,6835.888420,0.2200,0.905,0.692,-14.143,0.199,111.292,0.943
1,Kurtis Blow,Kurtis Blow,5TvcggVu3s2P1fMAS8BupX,The Breaks,1980,./data/The Breaks.mp3,1.057499e-07,6.933380e-08,-1.520881e-07,-3.618482e-07,...,2574.581462,2369.193568,5234.249962,0.0987,0.910,0.576,-13.256,0.109,112.877,0.907
2,8th Wonder,The Sugarhill Gang,4UCkX8nrBlpxjrrEqtb46a,Apache,1981,./data/Apache.mp3,-1.414954e-07,1.606020e-08,-4.006917e-08,-1.196751e-08,...,2535.741911,2907.329677,5791.839284,0.3320,0.827,0.928,-7.266,0.133,115.023,0.261
3,8th Wonder,The Sugarhill Gang,6m0wOLtrzaNwMofVVJTJqs,8th Wonder,1981,./data/8th Wonder.mp3,3.262833e-07,-1.683564e-08,-7.625627e-08,-1.203945e-07,...,2537.327894,3135.834750,5981.420399,0.2980,0.871,0.937,-8.536,0.195,106.494,0.675
4,Planet Rock,Afrika Bambaataa,3uy0jtkM8QYVTsBazkli1x,Planet Rock,1982,./data/Planet Rock.mp3,-8.671024e-08,7.907207e-08,-5.019072e-08,-2.139210e-08,...,2803.020324,3522.282559,6855.145450,0.0110,0.915,0.807,-9.636,0.130,127.350,0.708


In [83]:
# Looking to see if tempo has anything to do with years (i.e. are faster or slower tempos more prevalent in different
# time periods)?
subsets = musics[['mfcc_0','mfcc_1','mfcc_2','mfcc_3','mfcc_4','mfcc_5','mfcc_6','mfcc_7','mfcc_8','mfcc_9','mfcc_10','mfcc_11']]
mfccs = subsets.to_numpy().tolist()
# Transpose
#mfccs = list(map(list, zip(*mfccs)))
len(mfccs)
mfccs

[[1.8908110632764877e-08,
  -3.80420175360996e-07,
  -2.639254432779125e-07,
  -6.431759658198644e-08,
  -1.0801524297221476e-07,
  -1.7810402042073292e-08,
  -4.8014591413902956e-08,
  7.261296275373753e-08,
  1.1977847691468926e-07,
  8.293486075672263e-08,
  -7.197732543318125e-08,
  -4.6710475487543584e-08],
 [1.057498906220644e-07,
  6.933380092277731e-08,
  -1.5208810566491593e-07,
  -3.618482367073738e-07,
  6.409502617543694e-08,
  1.0533900507425642e-07,
  1.1406002897729194e-07,
  1.8733977924512146e-07,
  -1.244920895260293e-07,
  1.0643959313938467e-07,
  -1.5750370607747755e-07,
  1.3284184774420282e-07],
 [-1.4149540561447793e-07,
  1.606020205713321e-08,
  -4.006916753951373e-08,
  -1.1967512314470241e-08,
  2.808908732276905e-07,
  -8.390937722424496e-08,
  2.2512381292472128e-07,
  -1.7690622300392533e-08,
  -3.454570318695005e-08,
  1.5000613018401054e-07,
  4.7740652320271686e-08,
  5.36079491908481e-09],
 [3.26283270624117e-07,
  -1.6835636884593438e-08,
  -7.625627

In [84]:
features = mfccs
labels = musics['year'].tolist()

In [85]:
model = sklearn.linear_model.LogisticRegression()

In [86]:
model.fit(features,labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [87]:
import librosa
# In order to test the regression model, I will try it on some songs. I will be using our previously utilized 
ss = sklearn.preprocessing.StandardScaler()
def filename_to_features(fname):
    song, rate = librosa.load(fname)

    # 12 MFCC's
    mfccs = librosa.feature.mfcc(y = song, sr = rate, n_mfcc = 12).T
    mfcc_scaled = ss.fit_transform(X = mfccs)
    mfcc_series = pd.DataFrame(mfcc_scaled).mean()

    # 12 chroma features
    chroma = librosa.feature.chroma_stft(y = song, sr = rate)
    chroma_scaled = ss.fit_transform(X = chroma)
    chroma_series = pd.DataFrame(mfcc_scaled).mean()

    zcr = pd.Series(np.mean(librosa.feature.zero_crossing_rate(y = song)))
    bandwidth = pd.Series(np.mean(librosa.feature.spectral_bandwidth(y = song)))
    centroid = pd.Series(np.mean(librosa.feature.spectral_centroid(y = song)))
    rolloff = pd.Series(np.mean(librosa.feature.spectral_rolloff(y = song)))
    return pd.concat([mfcc_series, chroma_series, zcr, bandwidth, centroid, rolloff]).reset_index(drop = True)

In [88]:
# For the tests, I am going to try two different songs, one will be from the 1980s and one will be from the modern era 
#(2015-2020). The first song I will try is The Message by Grandmaster Flash and the Furious Five, regarded
#as one of the top hip-hop songs of the 80s. The year it was released was 1982.

message_features = filename_to_features('../data/message.mp3')

/Users/rakesh/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [89]:
message_mfccs = [message_features[0:12].tolist()]
message_mfccs

[[5.019176185783181e-08,
  1.299422507372583e-07,
  -1.7138839236707781e-09,
  -1.394677866528582e-07,
  -8.52012149721304e-08,
  -1.425027096324527e-09,
  1.217165674916032e-07,
  -5.0322711331318715e-08,
  3.411591720237084e-08,
  -9.686332624880833e-09,
  9.189498939576879e-08,
  3.2194055421541634e-08]]

In [96]:
model.predict(message_mfccs)

array([1992])

In [97]:
# So it predicted 1992 on a 1982 song. In order to quickly make sure it was not returning the same results for ANY
# song that was fed into the model, I decided to try another song before tweaking the model. This time, I used a 
# popular song from 2016, Jumpman by Drake and Future.

jumpman_features = filename_to_features('../data/jumpman.mp3')

/Users/rakesh/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [98]:
jumpman_mfccs = [jumpman_features[0:12].tolist()]
jumpman_mfccs

[[8.588823874333684e-08,
  -1.21068751468556e-07,
  -2.609560851851711e-07,
  -1.8458912620644696e-07,
  6.798580898248474e-08,
  1.812502432585461e-07,
  6.871194813129478e-08,
  -2.4023731892697242e-09,
  -4.723140420992422e-07,
  -4.166828215801388e-09,
  7.129754919787956e-08,
  9.697716052414762e-08]]

In [99]:
model.predict(jumpman_mfccs)
# This seems to indicate our training set isn't thorough enough. 

array([1996])

In [100]:
# To see if it would predict anything to be in the 2000s, I tried one more song: ZEZE by Kodak Black, Offset,
# and Travis Scott. 

zeze_features = filename_to_features('../data/zeze.mp3')

/Users/rakesh/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


In [102]:
zeze_mfccs = [zeze_features[0:12].tolist()]
zeze_mfccs

[[2.1652550685757888e-07,
  4.2068091232749794e-08,
  1.3325448833256814e-07,
  2.1818220830027713e-07,
  -6.124908225046966e-08,
  -2.8858613632110064e-07,
  -1.8406400670301082e-07,
  -1.0539112338392442e-07,
  -5.27906678371437e-08,
  2.8029031540199867e-08,
  3.266781334332336e-08,
  1.0550157014677097e-07]]

In [103]:
model.predict(zeze_mfccs)

array([1992])